In [1]:
# Import Libraries
import os
import glob
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from PIL import Image
import numpy as np

# Define Paths
dataset_dir = "/kaggle/input/imagesoasis/Data"  # Root folder containing class folders
# Adjust class names to match actual folder names
classes = ["Mild Dementia", "Moderate Dementia", "Non Demented", "Very mild Dementia"]

# Load Data with Correct Folder Names
image_paths, labels = [], []
for class_label, class_name in enumerate(classes):
    class_dir = os.path.join(dataset_dir, class_name)
    if not os.path.exists(class_dir):
        print(f"Error: Folder {class_dir} does not exist.")
        continue
    files = glob.glob(f"{class_dir}/*.jpg")  # Adjust extension if needed
    print(f"Class: {class_name}, Files Found: {len(files)}")  # Debug: Count files
    for file_path in files:
        image_paths.append(file_path)
        labels.append(class_label)

# Proceed with the pipeline if files are found
if len(image_paths) == 0:
    raise ValueError("No images found. Check dataset folder names or file paths.")

Class: Mild Dementia, Files Found: 5002
Class: Moderate Dementia, Files Found: 488
Class: Non Demented, Files Found: 67222
Class: Very mild Dementia, Files Found: 13725


In [2]:
# Split Dataset
train_paths, test_paths, train_labels, test_labels = train_test_split(
    image_paths, labels, test_size=0.2, random_state=42, stratify=labels
)

print(f"Total images: {len(image_paths)}")
print(f"Training images: {len(train_paths)}, Testing images: {len(test_paths)}")

Total images: 86437
Training images: 69149, Testing images: 17288


In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelBinarizer

# Normalize image paths and labels
train_paths = np.array(train_paths)
test_paths = np.array(test_paths)

# One-hot encode the labels
label_binarizer = LabelBinarizer()
train_labels = label_binarizer.fit_transform(train_labels)
test_labels = label_binarizer.transform(test_labels)

# Preprocessing Function for Images
IMG_HEIGHT, IMG_WIDTH = 224, 224

def preprocess_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [IMG_HEIGHT, IMG_WIDTH])
    image = image / 255.0  # Normalize to [0, 1]
    return image

# Load Dataset
def load_dataset(image_paths, labels):
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))
    dataset = dataset.map(lambda x, y: (preprocess_image(x), y))
    return dataset

# Prepare Train and Test Datasets
train_dataset = load_dataset(train_paths, train_labels)
test_dataset = load_dataset(test_paths, test_labels)

# Batch and Shuffle the Datasets
BATCH_SIZE = 32
train_dataset = train_dataset.shuffle(buffer_size=1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

2025-04-22 00:57:55.811859: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745283476.013474      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745283476.069884      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
I0000 00:00:1745283489.033454      31 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1745283489.034242      31 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability:

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models, callbacks
import os

base_model = ResNet50(include_top=False, weights='imagenet', input_shape=(224, 224, 3), pooling='avg')
for layer in base_model.layers:
    layer.trainable = False
for layer in base_model.layers[-2:]:
    layer.trainable = True

# Build model
model = models.Sequential([
    base_model,
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(4, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks
checkpoint_path = "best_model.keras"

callbacks_list = [
    callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    callbacks.ModelCheckpoint(checkpoint_path, monitor='val_loss', save_best_only=True, verbose=1)
]

# Train model
model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=30,
    callbacks=callbacks_list
)

Epoch 1/30
2161/2161 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - accuracy: 0.7736 - loss: 0.7195
Epoch 1: val_loss improved from inf to 0.66109, saving model to best_model.keras
2161/2161 ━━━━━━━━━━━━━━━━━━━━ 250s 109ms/step - accuracy: 0.7736 - loss: 0.7195 - val_accuracy: 0.7777 - val_loss: 0.6611
Epoch 2/30
2161/2161 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - accuracy: 0.7788 - loss: 0.6626
Epoch 2: val_loss improved from 0.66109 to 0.63427, saving model to best_model.keras
2161/2161 ━━━━━━━━━━━━━━━━━━━━ 227s 105ms/step - accuracy: 0.7788 - loss: 0.6626 - val_accuracy: 0.7777 - val_loss: 0.6343
Epoch 3/30
2161/2161 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - accuracy: 0.7788 - loss: 0.6481
Epoch 3: val_loss improved from 0.63427 to 0.62775, saving model to best_model.keras
2161/2161 ━━━━━━━━━━━━━━━━━━━━ 227s 105ms/step - accuracy: 0.7788 - loss: 0.6481 - val_accuracy: 0.7777 - val_loss: 0.6278
Epoch 4/30
2161/2161 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - accuracy: 0.7792 - loss: 0.6447
Epoch 4: val_loss improv

In [5]:
model2 = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(len(classes), activation='softmax')  
    ])

model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks
checkpoint_path = "best_model2.keras"

callbacks_list = [
    callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    callbacks.ModelCheckpoint(checkpoint_path, monitor='val_loss', save_best_only=True, verbose=1)
]

# Train model
model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=30,
    callbacks=callbacks_list
)


Epoch 1/30


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


2161/2161 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - accuracy: 0.7787 - loss: 0.6342
Epoch 1: val_loss improved from inf to 0.61043, saving model to best_model2.keras
2161/2161 ━━━━━━━━━━━━━━━━━━━━ 227s 105ms/step - accuracy: 0.7787 - loss: 0.6342 - val_accuracy: 0.7777 - val_loss: 0.6104
Epoch 2/30
2161/2161 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - accuracy: 0.7792 - loss: 0.6334
Epoch 2: val_loss did not improve from 0.61043
2161/2161 ━━━━━━━━━━━━━━━━━━━━ 226s 104ms/step - accuracy: 0.7792 - loss: 0.6334 - val_accuracy: 0.7777 - val_loss: 0.6200
Epoch 3/30
2161/2161 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - accuracy: 0.7791 - loss: 0.6345
Epoch 3: val_loss improved from 0.61043 to 0.60496, saving model to best_model2.keras
2161/2161 ━━━━━━━━━━━━━━━━━━━━ 227s 105ms/step - accuracy: 0.7791 - loss: 0.6345 - val_accuracy: 0.7777 - val_loss: 0.6050
Epoch 4/30
2161/2161 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - accuracy: 0.7783 - loss: 0.6373
Epoch 4: val_loss did not improve from 0.60496
2161/2161 ━━━━━━━━━━━━━━